# Hartree-Fock - Electron-Electron repulsion

The Fock operator is 

$$\hat{F}(\vec{r}) = \hat{h}(\vec{r}) +  \sum_{j=1}^{N} \left( \hat{J}_j(\vec{r}) - \hat{K}_j(\vec{r}) \right)$$ 

In a more general way, we can write the Coulomb operator as 

$$\hat{J}_{ij}(\vec{r}) = \int d^3r' \varphi^*_i(\vec{r}') \frac{1}{|\vec{r}-\vec{r}'|} \varphi_j(\vec{r}') $$

$$J_{ijkm} = \int d^3r \varphi^*_k(\vec{r}) \hat{J}_{ij}(\vec{r}) \varphi_m(\vec{r})$$

In this notebook we will be working with the electron-electron potential 'matrix', namely

$$\boxed{J_{ijkm} = \int d^3r d^3r' \varphi^*_i(\vec{r}') \varphi^*_k(\vec{r}) \frac{1}{|\vec{r}-\vec{r}'|} \varphi_j(\vec{r}') \varphi_m(\vec{r})}$$



In [1]:
import numpy as np
from Utils import * # Import functions from the previous notebooks

Expanding in Gaussian primitives, 

\begin{align*}
J_{ijkm} &= \sum_{\mu\nu\rho\sigma} c_{i\mu} c_{\nu j} c_{k\rho} c_{\sigma m} \int d^3r_1 d^3r_2 \phi_\mu(\vec{r}_1) \phi_\rho(\vec{r}_2) \frac{1}{|\vec{r}_1-\vec{r}_2|} \phi_\nu(\vec{r}_1) \phi_\sigma(\vec{r}_2) \\
&= \sum_{\mu\nu\rho\sigma} c_{i\mu} c_{\nu j} c_{k\rho} c_{\sigma m} N_\mu N_\nu N_\rho N_\sigma \int d^3r_1 d^3r_2 \frac{1}{|\vec{r}_1-\vec{r}_2|} \exp\left\{ - \alpha_\mu (\vec{r}_1-\vec{r}_\mu)^2\right\} \exp\left\{ - \alpha_\nu (\vec{r}_1-\vec{r}_\nu)^2\right\} \exp\left\{ - \alpha_\rho (\vec{r}_2-\vec{r}_\rho)^2\right\} \exp\left\{ - \alpha_\sigma (\vec{r}_1-\vec{r}_\sigma)^2\right\} \\
&= \sum_{\mu\nu\rho\sigma} c_{i\mu} c_{\nu j} c_{k\rho} c_{\sigma m} N_\mu N_\nu N_\rho N_\sigma \exp\left\{ -\frac{\alpha_\mu \alpha_\nu}{\alpha_p} (\vec{r}_\mu - \vec{r}_\nu)^2 \right\} \exp\left\{ -\frac{\alpha_\rho \alpha_\sigma}{\alpha_q}(\vec{r}_\rho - \vec{r}_\sigma)^2 \right\} \int d^3r_1 d^3r_2 \frac{1}{|\vec{r}_1-\vec{r}_2|} \exp\left\{ - \alpha_p (\vec{r}_1-\vec{r}_p)^2\right\}  \exp\left\{ -\alpha_q (\vec{r}_2-\vec{r}_q)^2\right\}
\end{align*}

As well as in the electron-nuclei attraction, we use the Fourier transform and exactly the same steps (convert spatial integrals into dirac's deltas and solve only one integral in the $k$ coordinate, such that $F_0$ is defined). The result is 

$$\boxed{J_{ijkm} = \sum_{\mu\nu\rho\sigma} c_{i\mu} c_{\nu j} c_{k\rho} c_{\sigma m} N_\mu N_\nu N_\rho N_\sigma \exp\left\{ -\frac{\alpha_\mu \alpha_\nu}{\alpha_p} (\vec{r}_\mu - \vec{r}_\nu)^2 \right\} \exp\left\{ -\frac{\alpha_\rho \alpha_\sigma}{\alpha_q}(\vec{r}_\rho - \vec{r}_\sigma)^2 \right\} \frac{2 \pi^{5/2}}{(\alpha_p \alpha_q)(\alpha_p + \alpha_q)^{1/2}} F_0\left( \alpha_r (\vec{r}_p-\vec{r}_q)^2 \right) }$$

where $\boxed{\alpha_r = \frac{\alpha_p \alpha_q}{\alpha_p+\alpha_q}}$

In [2]:
def electron_electron_repulsion(molecule):
    
    
    n_basis = len(molecule) # Number of atoms 
    V_ee = np.zeros((n_basis,n_basis,n_basis,n_basis)) # Overlap between the atomic orbitals
            
    # This set of loops runs over 4 orbitals because of the interaction matrix    
    for i in range(n_basis):
        for j in range(n_basis):
            for k in range(n_basis):
                for l in range(n_basis):
                    
                    nprimitives_i = len(molecule[i])
                    nprimitives_j = len(molecule[j])
                    nprimitives_k = len(molecule[k])
                    nprimitives_l = len(molecule[l])
                    
                    # This set of loops runs over the primitives in which each orbital is decomposed
                    for ii in range(nprimitives_i):
                        for jj in range(nprimitives_j):
                            for kk in range(nprimitives_k):
                                for ll in range(nprimitives_l):
                                    
                                    # Normalization constants product
                                    N = molecule[i][ii].A * molecule[j][jj].A * molecule[k][kk].A * molecule[l][ll].A 
                                    # Coefficients of expansion product
                                    cicjckcl = molecule[i][ii].coeff * molecule[j][jj].coeff * molecule[k][kk].coeff * molecule[l][ll].coeff
                                    
                                
                                    α_μ, α_ν, α_ρ, α_σ =  molecule[i][ii].α, molecule[j][jj].α, molecule[k][kk].α, molecule[l][ll].α
                                    
                                    α_p = α_μ + α_ν
                                    α_q = α_ρ + α_σ
                                    α_r = α_p * α_q / (α_p + α_q) 
                                    
                                    r_μ, r_ν, r_ρ, r_σ = molecule[i][ii].coords, molecule[j][jj].coords, molecule[k][kk].coords, molecule[l][ll].coords
                                    r_p = (α_μ * r_μ + α_ν * r_ν)/α_p
                                    r_q = (α_ρ * r_ρ + α_σ * r_σ)/α_q
                                    
                                    
                                    R = r_p-r_q
                                    R1 = r_μ-r_ν
                                    R2 = r_ρ-r_σ
                                    
                                    Rsq = np.dot(R,R)
                                    R1sq = np.dot(R1,R1)
                                    R2sq = np.dot(R2,R2)
                                    
                                    term_p = np.exp(-(α_μ*α_ν/α_p)*R1sq)
                                    term_q = np.exp(-(α_ρ*α_σ/α_q)*R2sq)
                                    
                                    V_ee[i,j,k,l] += N * cicjckcl * term_p * term_q * (2*np.pi**(5/2) / np.sqrt((α_p+α_q)))/(α_p*α_q) * F_0(α_r * Rsq)                                    
                                    
    return V_ee

In [4]:
# STO-3G BASIS FOR 1S ORBITALS
# Primitive gaussians of the first hydrogen
H1_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[0,0,0])
H1_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[0,0,0])
H1_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[0,0,0])

# Primitive gaussians of the second hydrogen
H2_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[0,0,1.4])
H2_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[0,0,1.4])
H2_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[0,0,1.4])

# Atomic orbitals
H1_1s = [H1_pgaussian1a, H1_pgaussian1b, H1_pgaussian1c]
H2_1s = [H2_pgaussian1a, H2_pgaussian1b, H2_pgaussian1c]

# Molecule
molecule = [H1_1s, H2_1s] 
print("STO-3G for 1s orbitals in H2")

# Overlap matrix
print("\n Overlap: \n",overlap(molecule))

# Kinetic energy matrix
print("\n Kinetic Energy: \n",kinetic(molecule))

# Electron-Nuclei attraction
atom_coordinates = np.array([[0,0,0],[0,0,1.4]])
Z = [1,1]
print("\n Electron-Nuclei attraction: \n", electron_nuclear_attraction(molecule,atom_coordinates,Z))

# Electron - Electron repulsion
print("\n Electron-Electron repulsion: \n", electron_electron_repulsion(molecule))

STO-3G for 1s orbitals in H2

 Overlap: 
 [[1.         0.65931821]
 [0.65931821 1.        ]]

 Kinetic Energy: 
 [[0.76003188 0.23645466]
 [0.23645466 0.76003188]]

 Electron-Nuclei attraction: 
 [[-1.88044089 -1.19483462]
 [-1.19483462 -1.88044089]]

 Electron-Electron repulsion: 
 [[[[0.77460594 0.44410766]
   [0.44410766 0.56967593]]

  [[0.44410766 0.29702854]
   [0.29702854 0.44410766]]]


 [[[0.44410766 0.29702854]
   [0.29702854 0.44410766]]

  [[0.56967593 0.44410766]
   [0.44410766 0.77460594]]]]
